In [1]:
pip install pandas

In [2]:
pip install matplotlib

In [3]:
pip install seaborn

In [4]:
pip install tensorflow

In [5]:
pip install keras

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from pathlib import Path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

In [10]:
import pandas as pd
import os

In [11]:
# Define the directory paths for real and fake images
real_images_dir = "/content/sample_data/data/Real"
fake_images_dir = "/content/sample_data/data/Fake"

In [12]:
# Gather file paths for real and fake images
real_image_paths = [os.path.join(real_images_dir, filename) for filename in os.listdir(real_images_dir) if filename.endswith('.jpg')]
fake_image_paths = [os.path.join(fake_images_dir, filename) for filename in os.listdir(fake_images_dir) if filename.endswith('.jpg')]

In [13]:
# Create labels for real and fake images
real_labels = ['real'] * len(real_image_paths)
fake_labels = ['fake'] * len(fake_image_paths)

In [14]:
# Combine file paths and labels
file_paths = real_image_paths + fake_image_paths
labels = real_labels + fake_labels

In [15]:
# Create DataFrame
image_df = pd.DataFrame({'Filepath': file_paths, 'Label': labels})

In [16]:
# Shuffle the DataFrame if needed
image_df = image_df.sample(frac=1).reset_index(drop=True)

In [17]:
# Display the DataFrame
print(image_df.head())

                                          Filepath Label
0   /content/sample_data/data/Fake/mid_46_1111.jpg  fake
1  /content/sample_data/data/Fake/mid_100_1110.jpg  fake
2    /content/sample_data/data/Real/real_00007.jpg  real
3    /content/sample_data/data/Real/real_00090.jpg  real
4   /content/sample_data/data/Fake/mid_47_1110.jpg  fake


In [18]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [19]:
# Define image preprocessing functions
def preprocess_image(filepath):
    # Load and preprocess image
    image = tf.keras.preprocessing.image.load_img(filepath, target_size=(256, 256))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

In [20]:
# Create train and test datasets
train_images = np.array([preprocess_image(filepath) for filepath in train_df['Filepath']])
train_labels = np.array(train_df['Label'])
test_images = np.array([preprocess_image(filepath) for filepath in test_df['Filepath']])
test_labels = np.array(test_df['Label'])

In [21]:
# Define the MesoNet model
model = Sequential([
    Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(8, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [22]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

In [29]:
print(train_images.dtype)
print(train_labels_encoded.dtype)

float32
int64


In [40]:
# Train the model
history = model.fit(train_images, train_labels_encoded, epochs=30, validation_split=0.2)

Epoch 1/30
9/9 [==============================] - 5s 581ms/step - loss: 0.5042 - accuracy: 0.8214 - val_loss: 0.6418 - val_accuracy: 0.6000
Epoch 2/30
9/9 [==============================] - 5s 510ms/step - loss: 0.4707 - accuracy: 0.8286 - val_loss: 0.6128 - val_accuracy: 0.7000
Epoch 3/30
9/9 [==============================] - 5s 584ms/step - loss: 0.4596 - accuracy: 0.8821 - val_loss: 0.6610 - val_accuracy: 0.6000
Epoch 4/30
9/9 [==============================] - 5s 509ms/step - loss: 0.4235 - accuracy: 0.8714 - val_loss: 0.6364 - val_accuracy: 0.6286
Epoch 5/30
9/9 [==============================] - 5s 573ms/step - loss: 0.4177 - accuracy: 0.8964 - val_loss: 0.6018 - val_accuracy: 0.7000
Epoch 6/30
9/9 [==============================] - 5s 517ms/step - loss: 0.4159 - accuracy: 0.9071 - val_loss: 0.6265 - val_accuracy: 0.6286
Epoch 7/30
9/9 [==============================] - 5s 523ms/step - loss: 0.3905 - accuracy: 0.9321 - val_loss: 0.6726 - val_accuracy: 0.5857
Epoch 8/30
9/9 [====

In [41]:
# Encode test_labels if they are in the string format
test_labels_encoded = label_encoder.transform(test_labels)

In [42]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

5/5 [==============================] - 1s 158ms/step - loss: 1.3290 - accuracy: 0.5467
Test Loss: 1.32901132106781
Test Accuracy: 0.54666668176651


In [43]:
# Make predictions on the test set
predictions = model.predict(test_images)
predictions = (predictions > 0.5).astype(int)

5/5 [==============================] - 1s 158ms/step


In [44]:
# Generate classification report and confusion matrix
print("Classification Report:")
print(classification_report(test_labels_encoded, predictions))
print("Confusion Matrix:")
print(confusion_matrix(test_labels_encoded, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.16      0.28        79
           1       0.51      0.97      0.67        71

    accuracy                           0.55       150
   macro avg       0.69      0.57      0.47       150
weighted avg       0.70      0.55      0.46       150

Confusion Matrix:
[[13 66]
 [ 2 69]]
